In [3]:
import lxml.etree as ET

from brainseg.svg.utils import is_point, save_element, is_line, points_to_numpy, \
    numpy_to_points, css_to_dict, dict_to_css
from brainseg.svg.parse import get_neuron_cat
import numpy as np

In [3]:
xml = ET.parse("/media/nathan/KINGSTON/Van_Essen_Project/svg input/M148 LHRH294seg.svg")
svg = xml.getroot()

# First objective is to extract the points

In [4]:
DYE_DICT = {
    "1": "FB",
    "2": "CTB555",
}

In [9]:
ls_neurons = []
last_comment = None
for x in svg.iterchildren():
    if is_point(x):
        cat = get_neuron_cat(last_comment.text)
        dye = DYE_DICT[cat]
        cx = x.attrib["x" if "x" in x.attrib else "cx"]
        cy = x.attrib["y" if "y" in x.attrib else "cy"]
        ls_neurons.append((dye, cx, cy))

    if isinstance(x, ET._Comment):
        last_comment = x
    else:
        last_comment = None


def neurons_to_str(ls, id_slice):
    return "\n".join(
        [','.join(
            [str(i), x, y, "0", id_slice, dye]
        ) for i, (dye, x, y) in enumerate(ls)]
    )

def create_neuron_file_content(neurons, id_slice):
    str_neurons = neurons_to_str(neurons, id_slice)
    string = "\n".join([
        "CSVF-FILE,0,,,," ,
        "csvf-section-start,header,2,,," ,
        "tag,value,,,," ,
        "Caret-Version,5.64,,,," ,
        "encoding,COMMA_SEPARATED_VALUE_FILE,,,," ,
        "csvf-section-end,header,,,," ,
        "csvf-section-start,Cells,6,,," ,
        "Cell Number,X,Y,Z,Section,Name" ,
        str_neurons,
        "csvf-section-end,Cells,,,,"
    ])
    return string

create_neuron_file_content(ls_neurons, "294")[:10]

In [26]:
# should be quite straightforward


  #produce the svg files
  lapply (contour.svg, write, paste ( output.names[s], "_pial.svg", sep = "") , append=TRUE, ncolumns=1 , sep = "\n")
  lapply (wm.svg, write, paste (output.names[s], "_white.svg", sep = "") , append=TRUE, ncolumns=1 , sep = "\n")
  
  if (length (position.of.cla) != 0) lapply (cla.svg, write, paste (output.names[s], "_cla.svg", sep = ""), append=TRUE, ncolumns=1, sep = "\n")
  if (length (position.of.cla_ven) != 0) lapply (cla_ven.svg, write, paste (output.names[s], "_cla_ven.svg", sep = ""), append=TRUE, ncolumns=1, sep = "\n")
  if (length (position.of.cla_AchE) != 0) lapply (cla_AchE.svg, write, paste (output.names[s], "_cla_AchE.svg", sep = ""), append=TRUE, ncolumns=1, sep = "\n")
  if (length (position.of.cla_li) != 0) lapply (cla_li.svg, write, paste (output.names[s], "_cla_li.svg", sep = ""), append=TRUE, ncolumns=1, sep = "\n")
  
  if (length (position.of.pu) != 0) lapply (pu.svg, write, paste (output.names[s], "_pu.svg", sep = ""), append=TRUE, ncolumns=1, sep = "\n")
  if (length (position.of.amy) != 0) lapply (amy.svg, write, paste (output.names[s], "_amy.svg", sep = ""), append=TRUE, ncolumns=1, sep = "\n")
  
  if (l4 == "yes") if (length (position.of.L4) != 0) lapply (L4.svg, write, paste (output.names[s], "_Layer4.svg", sep = ""), append=TRUE, ncolumns=1, sep = "\n")
  if (partial == "yes") lapply (partial.svg, write, paste (output.names[s] , "_partial_plot_marks.svg", sep = "") , append=TRUE, ncolumns=1 , sep = "\n")
  #Pay attention to the correct name format required by Van Essen's team which can be recognized by the subsequent script. -YH
  

# Next try to see if we can add something

In [4]:
xml = ET.parse("/media/nathan/KINGSTON/Van_Essen_Project/svg input/M148 LHRH294seg.svg")
svg = xml.getroot()

In [10]:
svg = ET.fromstring(
r"""<?xml version="1.0" standalone="yes"?>
<!-- Generator by matlab -->
<svg
xmlns="http://www.w3.org/2000/svg"
xmlns:xlink="http://www.w3.org/1999/xlink"
xmlns:svg="http://www.w3.org/2000/svg"
Version="1.1"
viewBox="0 0 30000 30000"
preserveAspectRatio="xMidYMid meet">

</svg>
"""
)

In [11]:
p = ET.Element("polyline", dict(
    points="589,-6460 589,-6460 1113,-6531 1119,-6530 1125,-6530"
           "1131,-6530 1142,-6535 1685,-6704 1691,-6704 1698,-6709",
    style="stroke:rgb(0,248,255); fill:none; stroke-width:1"
))

In [12]:
svg.append(p)

In [13]:
save_element(svg, "test_add.svg")

In [14]:
et = ET.tostring(svg, pretty_print=True, xml_declaration=True, standalone='yes')
et

b'<?xml version=\'1.0\' encoding=\'ASCII\' standalone=\'yes\'?>\n<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:svg="http://www.w3.org/2000/svg" Version="1.1" viewBox="0 0 30000 30000" preserveAspectRatio="xMidYMid meet">\n\n<polyline points="589,-6460 589,-6460 1113,-6531 1119,-6530 1125,-65301131,-6530 1142,-6535 1685,-6704 1691,-6704 1698,-6709" style="stroke:rgb(0,248,255); fill:none; stroke-width:1"/></svg>\n'

# Export the lines

In [46]:
xml = ET.parse("/media/nathan/KINGSTON/Van_Essen_Project/svg input/M148 LHRH294seg.svg")
svg = xml.getroot()

ls_points = []
last_comment = None
for x in svg.iterchildren():
    if not is_line(x):
        continue
        
    style = css_to_dict(x.attrib["style"])
    style["stroke-width"] = "100"
    x.attrib["style"] = dict_to_css(style)
        
    ls_points.append(points_to_numpy(x.attrib["points"]))

all_points = np.vstack(ls_points)

min_x, min_y = np.min(all_points, axis=0).astype(int)

svg.attrib["viewBox"] = f"{min_x - 500} {min_y - 500} 60000 60000"
svg.attrib["width"] = "1000"
svg.attrib["height"] = "1000"

save_element(svg, "/media/nathan/KINGSTON/Van_Essen_Project/svg input/test_lines294.svg")